In [1]:
import requests
import json
# first neural network with keras tutorial
import pandas as pd
import pandas_ta as ta
import random
from collections import deque
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Flatten, Activation, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint, Callback, TensorBoard, LearningRateScheduler
import time
import statistics
from sklearn import preprocessing
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os.path
from numpy import loadtxt
import yfinance as yf
import math
pd.options.mode.chained_assignment = None  # default='warn'

In [14]:

API_KEY = '026A5D39-77F3-4565-9EE4-5F4AC52419C2'

url = 'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_BTC_USD/history?period_id=30MIN&limit=10000&time_start=2021-07-15T00:00:00&time_end=2022-03-03T00:00:00'
headers = {'X-CoinAPI-Key' : API_KEY}
response = requests.get(url, headers=headers)
info = json.loads(response.content)



In [15]:
info = json.loads(response.content)

In [16]:
len(info)

5

In [5]:
open = []; high = []; low = []; close = []; volume = []; trades = []; timestamp = []

In [6]:
for row in info:
    open.append(row['price_open'])
    high.append(row['price_high'])
    low.append(row['price_low'])
    close.append(row['price_close'])
    volume.append(row['volume_traded'])
    trades.append(row['trades_count'])
    ts = row['time_period_end']
    timestamp.append("{}-{}-{} {}:{}:{}".format(ts[8:10],ts[5:7],ts[0:4],ts[11:13],ts[14:16],ts[17:19]))

In [7]:
column_names = ['timestamp', 'open', 'high', 'low', 'close', 'volume','trades']
df = pd.DataFrame(columns = column_names)

In [8]:
df['timestamp'] = timestamp
df['open'] = open
df['high'] = high
df['low'] = low
df['close'] = close
df['volume'] = volume
df['trades'] = trades

In [10]:
df.to_csv('BTC-USD.csv')

In [9]:
df.tail(5)

,timestamp,open,high,low,close,volume,trades
3736,02-03-2022 22:00:00,43826.82,44115.17,43802.58,44103.07,33.175146,357
3737,02-03-2022 22:30:00,44088.91,44380.88,43899.15,44065.61,37.239887,355
3738,02-03-2022 23:00:00,44065.54,44244.40,43976.96,44111.43,20.838905,337
3739,02-03-2022 23:30:00,44071.53,44212.41,44038.02,44045.22,5.834884,264
3740,03-03-2022 00:00:00,44005.14,44065.35,43875.19,43936.72,4.707596,246


In [119]:
# add data points
df['open_per1'] = df.ta.percent_return(length=1, close = df['open'])*100
df['ema7'] = df.ta.ema(length=7, close = df['open'])
df['ema3'] = df.ta.ema(length=3, close = df['open'])
df['rsi7'] = df.ta.rsi(length=7, close = df['open'])/100
df['rsi14'] = df.ta.rsi(length=14, close = df['open'])/100

df['volume14'] = df.ta.rsi(length=14, close = df['volume'])/100
df['volume7'] = df.ta.rsi(length=7, close = df['volume'])/100

df = df.dropna()
# df.reset_index(inplace=True)

In [120]:
df.tail(5)

,timestamp,open,high,low,close,volume,trades,open_per1,ema10,ema3,williams7,rsi7,williams14,rsi14,volume14,volume7
3736,02-03-2022 22:00:00,43826.82,44115.17,43802.58,44103.07,33.175146,357,-0.014327,43844.321730,43803.773114,0.441814,0.472500,0.805173,0.472819,0.485418,0.480023
3737,02-03-2022 22:30:00,44088.91,44380.88,43899.15,44065.61,37.239887,355,0.598013,43888.792325,43946.341557,0.317925,0.596582,0.463973,0.528793,0.489337,0.490106
3738,02-03-2022 23:00:00,44065.54,44244.40,43976.96,44111.43,20.838905,337,-0.053007,43920.928266,44005.940779,0.347647,0.582332,0.419324,0.523456,0.473663,0.449110
3739,02-03-2022 23:30:00,44071.53,44212.41,44038.02,44045.22,5.834884,264,0.013593,43948.310399,44038.735389,0.462365,0.585294,0.407024,0.524780,0.459173,0.412302
3740,03-03-2022 00:00:00,44005.14,44065.35,43875.19,43936.72,4.707596,246,-0.150641,43958.643054,44021.937695,0.576324,0.536129,0.446397,0.507937,0.458039,0.409361


In [127]:
df.to_csv('BTC-USD.csv')